In [ ]:
!nvidia-smi

Fri Jun 17 23:41:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers
!pip install simpletransformers

In [ ]:
!gdown --id 1lSu7e7BmF1xUHIVMldWuKBh_10Qbsgi_ #multi_modal_classification_model
!mv "/content/multi_modal_classification_model.py" "/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/multi_modal_classification_model.py"

NOTE: Restart runtime after running previous cells

#Setup and Config

In [ ]:
!gdown --id 1_Zds0QWmbn7bYn3iGrS1gyTLv33ZmwzU

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1_Zds0QWmbn7bYn3iGrS1gyTLv33ZmwzU
To: /content/Memes-final.zip
100% 143M/143M [00:02<00:00, 49.7MB/s]


In [ ]:
!unzip "/content/Memes-final.zip"
!rm Memes-final.zip

In [ ]:
!mv Memes\ \(final\) Memes-final

In [ ]:
import sklearn
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from simpletransformers.classification import MultiModalClassificationModel, MultiModalClassificationArgs
import seaborn as sns

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
seed = 777

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

#Data

In [ ]:
df = pd.read_excel('/content/Memes-final/final.xlsx')
df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)

In [ ]:
df

,file_name,meme_type,hate_type,meme_type2,hate_type2,text
0,img24.jpg,hate_speech,ddd,0.0,1.0,انا نظري ضعيف ومش هخش الجيش اقسم بالله انت قناص
1,img28.jpg,hate_speech,gender,0.0,4.0,لما فيمينست تتابع هانتر وتعرف غون بيبحث ابوه و...
2,img10.jpg,hate_speech,gender,0.0,4.0,لا أحد النسويات
3,img7.jpg,hate_speech,gender,0.0,4.0,يعني لو قلعنا الحجاب وتعرينا هنطلع القمر أوماااال
4,img16.jpg,hate_speech,gender,0.0,4.0,فيمينيست عربية نفسي اطلع اوروبا حرية وتحرر و ح...
...,...,...,...,...,...,...
993,img1311.jpeg,hate_speech,ddd,0.0,1.0,برو لماذا ابدو كمكعب العشب في ماينكرافت
994,img1313.jpg,hate_speech,ddd,0.0,1.0,حيوانات مصابة بمتلازمة داون المنغولية
995,img1309.png,hate_speech,ddd,0.0,1.0,كنت غبي ولاتستطيع الاصطفاف ابق في المنزل
996,img1308.png,hate_speech,ddd,0.0,1.0,عندما تضغط على ايقاف الفلاش على الكاميرا خاصتك


In [ ]:
train_images = df.file_name
train_text = df.text
train_labels = df.hate_type2

In [ ]:
train_data = []

for i in range(len(df)):
  train_data.append([train_images[i], train_text[i], train_labels[i]])

In [ ]:
train_data = pd.DataFrame(train_data, columns=['images','text','labels'])
train_data

,images,text,labels
0,img24.jpg,انا نظري ضعيف ومش هخش الجيش اقسم بالله انت قناص,1.0
1,img28.jpg,لما فيمينست تتابع هانتر وتعرف غون بيبحث ابوه و...,4.0
2,img10.jpg,لا أحد النسويات,4.0
3,img7.jpg,يعني لو قلعنا الحجاب وتعرينا هنطلع القمر أوماااال,4.0
4,img16.jpg,فيمينيست عربية نفسي اطلع اوروبا حرية وتحرر و ح...,4.0
...,...,...,...
993,img1311.jpeg,برو لماذا ابدو كمكعب العشب في ماينكرافت,1.0
994,img1313.jpg,حيوانات مصابة بمتلازمة داون المنغولية,1.0
995,img1309.png,كنت غبي ولاتستطيع الاصطفاف ابق في المنزل,1.0
996,img1308.png,عندما تضغط على ايقاف الفلاش على الكاميرا خاصتك,1.0


In [ ]:
train_df, test_df = train_test_split(train_data, test_size=0.1, random_state=41)

In [ ]:
print(len(train_df))
print(len(test_df))

898
100


In [ ]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

In [ ]:
print(len(train_df))
print(len(test_df))

746
93


In [ ]:
model_args = MultiModalClassificationArgs(
    num_train_epochs=7,
    fp16=False,
    train_batch_size=8,
    learning_rate=3e-5,
    max_seq_length=32,
    save_best_model=True,
    overwrite_output_dir=True,
    save_model_every_epoch=False,
    gradient_accumulation_steps=1,
)

In [ ]:
model = MultiModalClassificationModel(
  'bert',
  'bert-base-multilingual-uncased',
  use_cuda=True,
  label_list=[0,1,2,3,4],
  args=model_args
) 

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.config.use_return_dict = False

In [ ]:
model.train_model(
        train_df, 
        image_path='/content/Memes-final/'
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 0 of 7:   0%|          | 0/94 [00:00<?, ?it/s]

Running Epoch 1 of 7:   0%|          | 0/94 [00:00<?, ?it/s]

Running Epoch 2 of 7:   0%|          | 0/94 [00:00<?, ?it/s]

...

Running Epoch 4 of 7:   0%|          | 0/94 [00:00<?, ?it/s]

Running Epoch 5 of 7:   0%|          | 0/94 [00:00<?, ?it/s]

Running Epoch 6 of 7:   0%|          | 0/94 [00:00<?, ?it/s]

INFO:simpletransformers.classification.multi_modal_classification_model: Training of bert model complete. Saved to outputs/.


(658, 0.7757510004400935)

In [ ]:
import sklearn

result, model_outputs = model.eval_model(
    test_df,
    acc=sklearn.metrics.accuracy_score,
    image_path='/content/Memes-final/',
)

Running Evaluation:   0%|          | 0/12 [00:00<?, ?it/s]

INFO:simpletransformers.classification.multi_modal_classification_model:{'mcc': 0.5573844184552927, 'acc': 0.6451612903225806, 'eval_loss': 1.3906876544157665}


In [ ]:
test_df

,images,text,labels
218,img240.jpg,وأخيرا السعودية تسمح للنساء بقيادة السيارة,2.0
319,img323.jpg,شاب البنات ولا مرة رح يعرفوا قديش صعب الواحد ي...,4.0
832,img1169.png,داعشي مسلم يقوم بشق عنق كافر اخر,0.0
962,img1310.png,الذي يجعلك تشعر باقصى درجات المحنة,2.0
455,img457.jpg,لجمع الاغراض المدرسية لجمع النقود لجمع العرب,3.0
...,...,...,...
123,img111.jpg,مواطن اردني يهاجر هو ايضا بعدما وصل لوجهته هل ...,2.0
361,img369.jpg,عندما تشعر بالوحدة ثم تشتري آسيويين من الشبكة ...,3.0
704,img1034.jpg,الزنوج البشرة اليبضاء,3.0
248,img268.png,الناس الي بتفكر انه النسوية هي كره للرجال,4.0


In [ ]:
# model_outputs

In [ ]:
_, preds = torch.max(torch.tensor(model_outputs), dim=1)

preds[0]

tensor(2)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

labels = np.array(test_df.labels)

print("F1 macro:   {}".format(round(f1_score(labels, preds, average="macro"), 3)))
print("Precision:   {}".format(precision_score(labels,preds, average="macro"), 3))
print("Recall:   {}".format(recall_score(labels,preds, average="macro"), 3))
print("F1 micro:   {}".format(round(f1_score(labels, preds, average="micro"), 3)))